# 必要ライブラリの読み込み

In [1]:
#ライブラリ読み込み
import numpy as np
import pandas as pd

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import mean_squared_error

from automllib.ensemble import LGBMRegressorCV
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import KFold
from tqdm import tqdm
import geocoder

pd.set_option('display.max_columns',200)
pd.set_option('display.max_rows',200)

In [2]:
import sklearn

print(sklearn.__version__) 

0.21.2


# 必要データの読み込み

In [3]:
#データ読み込み

#データ定義
data_definition = pd.read_csv('input/data_definition.txt',sep = "\t")

#trainデータ読み込み
train_genba = pd.read_csv('input/train_genba.tsv',sep = "\t")
train_goto = pd.read_csv('input/train_goto.tsv',sep = "\t")

#testデータ読み込み
test_genba = pd.read_csv('input/test_genba.tsv',sep = "\t")
test_goto = pd.read_csv('input/test_goto.tsv',sep = "\t")

#サンプルサブミッション
sample_submit = pd.read_csv('input/sample_submit.tsv',sep = "\t")

# #緯度経度情報
# city_position = pd.read_csv('input/city_position.csv')


# データの結合

In [4]:
train_data = pd.merge(train_goto,train_genba,on ='pj_no',how ='left')
test_data = pd.merge(test_goto,test_genba,on ='pj_no',how ='left')

In [5]:
#異常っぽい3つのデータを削除
train_data = train_data.drop([4663,5056,5823])

 # 前処理

In [6]:
train_data.shape

(6458, 158)

# 個別処理

In [7]:
train_data['city1'] = train_data['jukyo'].str.split('市') 
train_data['city1'] = train_data['city1'].map(lambda x: x[0])
train_data['city2'] = train_data['city1'].str.split('郡') 
train_data['city2'] = train_data['city2'].map(lambda x: x[0])
train_data['city3'] = train_data['city2'].str.split('町') 
train_data['city3'] = train_data['city3'].map(lambda x: x[0])

test_data['city1'] = test_data['jukyo'].str.split('市') 
test_data['city1'] = test_data['city1'].map(lambda x: x[0])
test_data['city2'] = test_data['city1'].str.split('郡') 
test_data['city2'] = test_data['city2'].map(lambda x: x[0])
test_data['city3'] = test_data['city2'].str.split('町') 
test_data['city3'] = test_data['city3'].map(lambda x: x[0])

category_columns = ['city1','city2','city3']

## 緯度経度追加

In [8]:
# city_position.rename(columns={'city':'city3'},inplace=True)

# train_data = pd.merge(train_data,city_position,how='left')
# test_data = pd.merge(test_data,city_position,how='left')

# # train_data['tokyoeki_distance'] = (train_data['longitude'] - 139.767125)**2 + (train_data['latitude'] - 35.681236)**2
# # test_data['tokyoeki_distance'] = (test_data['longitude'] - 139.767125)**2 + (test_data['latitude'] - 35.681236)**2

## 階とLDKに分割

In [9]:
train_data['プラン_分割後'] = train_data['levelplan'].str.split('/') 
train_data_plan = train_data[train_data['プラン_分割後'] .notnull()]
train_data_plan = train_data_plan.query('levelplan != "土地売り"')
train_data_plan['階数'] = train_data_plan['プラン_分割後'].map(lambda x: x[0])
train_data_plan['部屋数'] = train_data_plan['プラン_分割後'].map(lambda x: x[1])
train_data = pd.merge(train_data,train_data_plan[['id','階数','部屋数']],on ='id',how='left')
train_data = train_data.drop('プラン_分割後',axis=1)
#train_data['階数'] = train_data['階数'].mask(train_data['levelplan'] == '土地売り','土地売り')

In [10]:
test_data['プラン_分割後'] = test_data['levelplan'].str.split('/') 
test_data_plan = test_data[test_data['プラン_分割後'] .notnull()]
test_data_plan = test_data_plan.query('levelplan != "土地売り"')
test_data_plan['階数'] = test_data_plan['プラン_分割後'].map(lambda x: x[0])
test_data_plan['部屋数'] = test_data_plan['プラン_分割後'].map(lambda x: x[1])
test_data = pd.merge(test_data,test_data_plan[['id','階数','部屋数']],on ='id',how='left')
test_data = test_data.drop('プラン_分割後',axis=1)
#test_data['階数'] = test_data['階数'].mask(test_data['levelplan'] == '土地売り','土地売り')

In [11]:
category_columns += ['階数','部屋数']

# 数値型に関する処理

In [15]:
numeric_columns = data_definition.query('データ型 == "数値"')['項目名'].tolist()

In [16]:
#高相関や内容的に不要そうな列
drop_numeric_list = ['chiseki_kb_hb','kempei1','kempei2','yoseki1','yoseki2','tateuri_su','hy1f_date_su','road1_fi',
                     'road1_mg','road2_fi','road2_mg','road3_mg','road3_mg','road4_fi','road4_fi','road4_mg',
                     'mseki_yt_hb','tc_mseki_min_hb','fi4m_kyori','fi3m_kyori','tc_mseki_max_hb','tt_mseki_avg_hb','niwasaki']

train_data.drop(columns=drop_numeric_list,axis=1,inplace=True)
test_data.drop(columns=drop_numeric_list,axis=1,inplace=True)

KeyError: "['chiseki_kb_hb' 'kempei1' 'kempei2' 'yoseki1' 'yoseki2' 'tateuri_su'\n 'hy1f_date_su' 'road1_fi' 'road1_mg' 'road2_fi' 'road2_mg' 'road3_mg'\n 'road3_mg' 'road4_fi' 'road4_fi' 'road4_mg' 'mseki_yt_hb'\n 'tc_mseki_min_hb' 'fi4m_kyori' 'fi3m_kyori' 'tc_mseki_max_hb'\n 'tt_mseki_avg_hb' 'niwasaki'] not found in axis"

In [17]:
#路線価は最寄り駅から欠損値保管

In [18]:
#0を欠損に置換した方が良さそうな列
zero_impute_list = ['rosenka_hb','kijun_hb','mseki_rd_hb','tt_mseki_min_hb','tt_mseki_max_hb','tt_mseki']

for column in zero_impute_list:
    train_data[column].replace(0, np.nan,inplace=True)
    test_data[column].replace(0, np.nan,inplace=True)

In [19]:
#!!!補完によってむしろ精度が悪化した

# #路線価格は最寄り駅が同じデータを探して欠損値保管
# #trainとtestを縦に結合
# train_test_data = pd.concat([train_data.drop('keiyaku_pr',axis=1),test_data],axis=0).reset_index()

# f = lambda x: x.fillna(x.max())
# transformed = train_test_data[['eki_nm1','rosenka_hb']].groupby('eki_nm1').transform(f)

# train_rosenka = transformed.iloc[0:6461,:].reset_index()
# test_rosenka = transformed.iloc[6461:,:].reset_index()

# train_data =pd.concat([train_data.drop('rosenka_hb',axis=1),train_rosenka[['rosenka_hb']]],axis=1)
# test_data =pd.concat([test_data.drop('rosenka_hb',axis=1),test_rosenka[['rosenka_hb']]],axis=1)

In [20]:
#数値型のリスト最終的に
set1 = set(numeric_columns)
set2 = set(drop_numeric_list )

#set1とset2の積集合を得る
diff_set = set(set1) - set(set2)

#setをlistに戻すときは
numeric_columns=list(diff_set)

In [21]:
len(numeric_columns)

34

In [22]:
train_data.shape

(6458, 142)

# フラグ型に関する処理

In [23]:
flag_columns = data_definition.query('データ型 == "2値"')['項目名'].tolist()

In [24]:
len(flag_columns)

59

In [25]:
#○の処理
maru_flag_columns =['sho_conv','sho_super','sho_shoten','sho_market','shu_jutaku','shu_park','shu_shop','shu_factory','shu_hvline',
                    'shu_tower','shu_bochi','shu_sogi','shu_zoki','shu_kokyo','shu_highway','shu_kaido','shu_line_ari','shu_line_nashi',
                    'shu_soon','rs_e_kdate2','rs_e_kdate3','rs_e_parking','rs_e_zoki','rs_e_m_ari','rs_e_m_nashi','rs_e_tahata',
                    'rs_w_kdate2','rs_w_kdate3','rs_w_parking','rs_w_zoki','rs_w_m_ari','rs_w_m_nashi','rs_w_tahata','rs_s_kdate2',
                    'rs_s_kdate3','rs_s_parking','rs_s_zoki','rs_s_m_ari','rs_s_m_nashi','rs_s_tahata','rs_n_kdate2','rs_n_kdate3',
                    'rs_n_parking','rs_n_zoki','rs_n_m_ari','rs_n_m_nashi','rs_n_tahata']

for column in maru_flag_columns:
    train_data[column].replace('○', 1,inplace=True)
    train_data[column].fillna(0,inplace=True)
    test_data[column].replace('○', 1,inplace=True)
    test_data[column].fillna(0,inplace=True)
    
    
#有無の処理
um_flag_columns = ['kaoku_um','yheki_umu','chikukeikaku','keikakuroad']

for column in um_flag_columns:
    train_data[column].replace({'（有）':1,'（無）':0},inplace=True)
    test_data[column].replace({'（有）':1,'（無）':0},inplace=True)
    
    
#要不要の処理
you_flag_columns = ['yheki_yohi','kaihatsukyoka','t53kyoka','hokakyoka','fi4m_yohi','fi3m_yohi','bus_yohi','kinshijiko']
#kinshijikoはカテゴリだが、実際2値なのでこっち

for column in you_flag_columns:
    train_data[column].fillna(0,inplace=True)
    test_data[column].fillna(0,inplace=True)
    train_data[column].replace({'（要）':1,'（不要）':0,'（無）':0},inplace=True)
    test_data[column].replace({'（要）':1,'（不要）':0,'（無）':0},inplace=True)

#取引形態
train_data['kborjs'].replace({'公募':1,'公簿':1,'実測':0},inplace=True)
test_data['kborjs'].replace({'公募':1,'公簿':1,'実測':0},inplace=True)

#型をintに変換
for column in flag_columns :
    train_data[column] = train_data[column].astype(bool)
    test_data[column] = test_data[column].astype(bool)

In [26]:
#フラグ型の列の情報を作成
target_df = pd.DataFrame()

posi_nega_list = []
for column in flag_columns:
    temp_target_df = train_data.groupby(column).mean()[['keiyaku_pr']]
    count_df = train_data.groupby(column).count()[['keiyaku_pr']]
    
    if count_df.min().values>100:
        posi_nega_list.append(column)
        temp_target_df.rename(columns={'keiyaku_pr':column},inplace=True)
        temp_target_df = temp_target_df.T
        target_df = target_df.append(temp_target_df)
        
target_df['平均差'] = target_df.iloc[:,1]-target_df.iloc[:,0]
target_df['平均差'] = target_df['平均差'].mask(target_df['平均差']>0,1)
target_df['平均差'] = target_df['平均差'].mask(target_df['平均差']<0,-1)

train_data['posi_nega'] = 0
test_data['posi_nega'] = 0
for column in posi_nega_list:
    train_data['posi_nega'] = train_data['posi_nega'] +train_data[column] * target_df.query('index == @column')['平均差'].values[0]
    test_data['posi_nega'] = test_data['posi_nega'] +test_data[column] * target_df.query('index == @column')['平均差'].values[0]
    print(column)

kaoku_um
yheki_umu
yheki_yohi
kborjs
chikukeikaku
keikakuroad
kaihatsukyoka
t53kyoka
hokakyoka
fi4m_yohi
bus_yohi
sho_conv
sho_super
sho_shoten
sho_market
shu_jutaku
shu_park
shu_factory
shu_kaido
rs_e_kdate2
rs_e_parking
rs_e_m_nashi
rs_e_tahata
rs_w_kdate2
rs_w_parking
rs_w_m_nashi
rs_w_tahata
rs_s_kdate2
rs_s_kdate3
rs_s_parking
rs_s_m_nashi
rs_s_tahata
rs_n_kdate2
rs_n_parking
rs_n_m_nashi
rs_n_tahata


In [28]:
#駅への移動手段×距離
train_data['bus_kyori1'] = np.nan
train_data['bus_kyori1'] = train_data['bus_kyori1'].mask(train_data['bas_toho1'] == 'バス',train_data['eki_kyori1'])

train_data['toho_kyori1'] = np.nan
train_data['toho_kyori1'] = train_data['toho_kyori1'].mask(train_data['bas_toho1'] == '徒歩',train_data['eki_kyori1'])

train_data['car_kyori1'] = np.nan
train_data['car_kyori1'] = train_data['car_kyori1'].mask(train_data['bas_toho1'] == '車',train_data['eki_kyori1'])

train_data['bus_kyori2'] = np.nan
train_data['bus_kyori2'] = train_data['bus_kyori2'].mask(train_data['bas_toho2'] == 'バス',train_data['eki_kyori2'])

train_data['toho_kyori2'] = np.nan
train_data['toho_kyori2'] = train_data['toho_kyori2'].mask(train_data['bas_toho2'] == '徒歩',train_data['eki_kyori2'])

train_data['car_kyori2'] = np.nan
train_data['car_kyori2'] = train_data['car_kyori2'].mask(train_data['bas_toho2'] == '車',train_data['eki_kyori2'])

In [29]:
#駅への移動手段×距離
test_data['bus_kyori1'] = np.nan
test_data['bus_kyori1'] = test_data['bus_kyori1'].mask(test_data['bas_toho1'] == 'バス',test_data['eki_kyori1'])

test_data['toho_kyori1'] = np.nan
test_data['toho_kyori1'] = test_data['toho_kyori1'].mask(test_data['bas_toho1'] == '徒歩',test_data['eki_kyori1'])

test_data['car_kyori1'] = np.nan
test_data['car_kyori1'] = test_data['car_kyori1'].mask(test_data['bas_toho1'] == '車',test_data['eki_kyori1'])

test_data['bus_kyori2'] = np.nan
test_data['bus_kyori2'] = test_data['bus_kyori2'].mask(test_data['bas_toho2'] == 'バス',test_data['eki_kyori2'])

test_data['toho_kyori2'] = np.nan
test_data['toho_kyori2'] = test_data['toho_kyori2'].mask(test_data['bas_toho2'] == '徒歩',test_data['eki_kyori2'])

test_data['car_kyori2'] = np.nan
test_data['car_kyori2'] = test_data['car_kyori2'].mask(test_data['bas_toho2'] == '車',test_data['eki_kyori2'])

In [30]:
#近隣の買い物施設数
train_data['kaimono'] = train_data['sho_conv'] + train_data['sho_market'] + train_data['sho_shoten'] + train_data['sho_super']

#基準地価×建物面積
train_data['kijun_tatemenseki'] = train_data['kijun_hb'] * train_data['tt_mseki'] 

#公示地価×建物面積
train_data['koji_tatemenseki'] = train_data['koji_hb'] * train_data['tt_mseki'] 

#基準地価×土地面積
train_data['kijun_totimenseki'] = train_data['kijun_hb'] * train_data['tc_mseki'] 

#公示地価×土地面積
train_data['koji_totimenseki'] = train_data['koji_hb'] * train_data['tc_mseki'] 

#現場に占める割合
train_data['menseki_hi'] = train_data['tc_mseki']/train_data['chiseki_js_hb']

/opt/conda/lib/python3.7/site-packages/pandas/core/computation/expressions.py:183: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [31]:
#近隣の買い物施設数
test_data['kaimono'] = test_data['sho_conv'] + test_data['sho_market'] + test_data['sho_shoten'] + test_data['sho_super']

#基準地価×建物面積
test_data['kijun_tatemenseki'] = test_data['kijun_hb'] * test_data['tt_mseki'] 

#公示地価×建物面積
test_data['koji_tatemenseki'] = test_data['koji_hb'] * test_data['tt_mseki'] 

#基準地価×土地面積
test_data['kijun_totimenseki'] = test_data['kijun_hb'] * test_data['tc_mseki'] 

#公示地価×土地面積
test_data['koji_totimenseki'] = test_data['koji_hb'] * test_data['tc_mseki'] 

#現場に占める割合
test_data['menseki_hi'] = test_data['tc_mseki']/test_data['chiseki_js_hb']

In [32]:
train_data.shape

(6458, 155)

# カテゴリ、テキスト型に対する処理

In [33]:
category_columns = category_columns + data_definition.query('データ型 == "カテゴリ" or データ型 == "テキスト"')['項目名'].tolist()

In [34]:
len(category_columns)

48

In [35]:
#LightGBMでのカテゴリ扱いをする変数を指定
for column in category_columns:
    train_data[column] = train_data[column].astype('category')
    test_data[column] = test_data[column].astype('category')

In [38]:
#カウントエンコーディング

#trainとtestを縦に結合
train_test_data = pd.concat([train_data.drop('keiyaku_pr',axis=1),test_data],axis=0)


#trainとテストをくっつけてカウントエンコーディング
train_test_count = train_test_data[category_columns+['id']].copy()
count_encoded_columns=[]

for column in category_columns: 
    train_test_count[column] = train_test_data.groupby(column)['id'].transform('count')
    train_test_count.rename(columns={column:'count_'+column},inplace=True)
    count_encoded_columns.append('count_'+column)
    
train_test_count.drop('id',axis=1,inplace =True)

#trainとtestを分離
train_count_encoded = train_test_count.iloc[0:6458,:]
test_count_encoded = train_test_count.iloc[6458:,:]

#もともとのカテゴリ型の列を削除し、カウントエンコーディング結果を結合
#train_data.drop(columns=category_columns,axis=1,inplace=True)
#test_data.drop(columns=category_columns,axis=1,inplace=True)

train_data =pd.concat([train_data,train_count_encoded],axis=1)
test_data =pd.concat([test_data,test_count_encoded],axis=1)

In [39]:
train_data.shape

(6458, 203)

In [40]:
#ターゲットエンコーディングを行う列(リークを防ぐためにtrainの最小頻度でも10回)
target_columns = [
'city3',
'yoto1',
'josui',
'gesui',
'gas',
'usui',
'hw_status',
'rosen_nm1',
'eki_nm1',
'road_st',
'setsudo_kj',
'jigata',
'hiatari']

In [41]:
# # #カテゴリと集約するターゲットを結合
# train_categorical_target = train_data[target_columns+['keiyaku_pr']]

# #ターゲットエンコーディングの元になるデータフレーム
# train_target_encoded = train_data[target_columns]
# test_target_encoded = test_data[target_columns]

# #カテゴリごとに値の平均値を取る
# for column in tqdm(target_columns):
#     #カウントとへいきんの集約を実施
#     target_matrix = train_categorical_target.groupby(column).mean()[['keiyaku_pr']]
#     target_matrix.rename(columns={'keiyaku_pr':'target_'+column},inplace=True)
#     count_matrix = train_categorical_target.groupby(column).count()[['keiyaku_pr']]
#     count_matrix.rename(columns={'keiyaku_pr':'count_t_'+column},inplace=True)
#     target_matrix = pd.merge(count_matrix,target_matrix,on=column,how='inner')
    
#     #リークの影響を薄めるために10回以上出現したカテゴリのみを抽出
#     column_name = 'count_t_'+column
#     target_matrix = target_matrix[target_matrix[column_name]>10]
    
#     #trainとtestに集約した値を結合
#     train_target_encoded =pd.merge(train_target_encoded,target_matrix,on=column,how='left')
#     train_target_encoded.drop(column,axis=1,inplace=True)

#     test_target_encoded =pd.merge(test_target_encoded,target_matrix,on=column,how='left')
#     test_target_encoded.drop(column,axis=1,inplace=True)

# train_data =pd.concat([train_data,train_target_encoded],axis=1)
# test_data =pd.concat([test_data,test_target_encoded],axis=1)

In [42]:
test_data.shape

(4273, 202)

# 入力データの整形

In [43]:
# set1 = set(category_columns)
# set2 = set(important_117)

# #set1とset2の積集合を得る
# and_set = set1.intersection(set2)

# #setをlistに戻すときは
# important_categorical_columns=list(and_set)

In [44]:
important = ['tc_mseki',
 'tt_mseki',
 'levelplan',
 'fukuin',
 'magutchi',
 'setsudo_hi',
 'setsudo_kj',
 'jigata',
 'hiatari',
 'garage',
 'kobetsu1',
 'jukyo',
 'chiseki_js_hb',
 'yoto1',
 'josui',
 'gesui',
 'gas',
 'usui',
 'tochiuri_su',
 'joken_su',
 'hy2f_date_su',
 'hy3f_date_su',
 'road1_hk',
 'road1_sb',
 'road2_hk',
 'road2_sb',
 'road3_hk',
 'road3_sb',
 'road3_fi',
 'road4_sb',
 'kaoku_um',
 'kaoku_hb',
 'yheki_kotei',
 'yheki_umu',
 'yheki_yohi',
 'gk_sho_kyori',
 'gk_chu_kyori',
 'kborjs',
 'hw_status',
 'toshikuiki1',
 'kodochiku',
 'chikukeikaku',
 'keikakuroad',
 'kaihatsukyoka',
 't53kyoka',
 'hokakyoka',
 'bokachiiki',
 'minmenseki',
 'hokakisei1',
 'kinshijiko',
 'rosenka_hb',
 'koji_hb',
 'kijun_hb',
 'mseki_rd_hb',
 'mseki_dp_hb',
 'tt_mseki_min_hb',
 'tt_mseki_max_hb',
 'tc_mseki_avg_hb',
 'fi4m_yohi',
 'bus_yohi',
 'bus_hon',
 'sho_conv',
 'sho_super',
 'sho_shoten',
 'sho_market',
 'shu_jutaku',
 'shu_park',
 'shu_shop',
 'shu_bochi',
 'shu_zoki',
 'shu_kokyo',
 'shu_kaido',
 'shu_line_ari',
 'shu_line_nashi',
 'gk_yoc_tm',
 'gk_sho_tm',
 'gk_chu_tm',
 'rs_e_kdate2',
 'rs_e_kdate3',
 'rs_e_parking',
 'rs_e_m_nashi',
 'rs_e_tahata',
 'rs_w_kdate2',
 'rs_w_parking',
 'rs_w_m_nashi',
 'rs_w_tahata',
 'rs_s_kdate2',
 'rs_s_kdate3',
 'rs_s_parking',
 'rs_s_zoki',
 'rs_s_m_nashi',
 'rs_s_tahata',
 'rs_n_kdate2',
 'rs_n_parking',
 'rs_n_m_nashi',
 'rs_n_tahata',
 'rosen_nm1',
 'eki_nm1',
 'bas_toho1',
 'eki_kyori1',
 'teiho1',
 'bas_toho2',
 'eki_kyori2',
 'city1',
 'city2',
 'city3',
 'longitude',
 'latitude',
 'posi_nega',
 'bus_kyori1',
 'toho_kyori1',
 'car_kyori1',
 'toho_kyori2',
 'kaimono',
 'kijun_tatemenseki',
 'koji_tatemenseki',
 'kijun_totimenseki',
 'koji_totimenseki',
 'menseki_hi',
 'count_city1',
 'count_city2',
 'count_city3',
 'count_jukyo',
 'count_yoto1',
 'count_yoto2',
 'count_gesui',
 'count_gas',
 'count_usui',
 'count_road1_hk',
 'count_road1_sb',
 'count_road2_hk',
 'count_road2_sb',
 'count_road3_hk',
 'count_road3_sb',
 'count_road4_hk',
 'count_hw_status',
 'count_toshikuiki1',
 'count_kodochiku',
 'count_bokachiiki',
 'count_hokakisei1',
 'count_hokakisei2',
 'count_hokakisei3',
 'count_kinshijiko',
 'count_rosen_nm1',
 'count_eki_nm1',
 'count_bas_toho1',
 'count_bastei_nm1',
 'count_rosen_nm2',
 'count_eki_nm2',
 'count_bas_toho2',
 'count_levelplan',
 'count_road_st',
 'count_setsudo_hi',
 'count_setsudo_kj',
 'count_jigata',
 'count_hiatari',
 'count_kobetsu1',
 'count_kobetsu2',
 'count_kobetsu3']

In [45]:
# train_input = train_data[important+['id','pj_no','keiyaku_pr']]
# test_input = test_data[important+['id','pj_no']]

train_input = train_data
test_input = test_data

In [46]:
test_input.shape

(4273, 202)

In [0]:
train_input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6458 entries, 0 to 6457
Columns: 205 entries, id to count_kobetsu4
dtypes: bool(60), category(48), float64(69), int64(27), object(1)
memory usage: 5.6+ MB


In [0]:
test_data.shape

(4273, 204)

# データ整形

In [0]:
#必要な列だけ残す
X = train_input.drop(['id','pj_no','keiyaku_pr'],axis = 1)
X_test = test_input.drop(['id','pj_no'],axis = 1)
y = train_data[['keiyaku_pr']]

Parameter_df =pd.DataFrame()
MAPE_df = pd.DataFrame()
MAPE_train_df =pd.DataFrame()
Submit_df = test_data[['id']].copy(deep = True)
Submit_df['MAPE_sum'] = 0

#グループ分けの定義
groups = train_input['pj_no'].array


#gkf = GroupShuffleSplit(test_size =0.2,random_state=0)
kf = KFold(n_splits= 5,shuffle=True,random_state=0)

i = 0
#for train,valid  in tqdm(gkf.split(X, y,groups=groups)):
for train,valid in tqdm(kf.split(X,y)):
    X_train = X.iloc[train.tolist()]
    X_valid = X.iloc[valid.tolist()]
    y_train = y.iloc[train.tolist()]
    y_valid = y.iloc[valid.tolist()]

#     # 学習データと検証データに分割する
#     #X_train, X_valid, y_train, y_valid = train_test_split(X, y,test_size = 0.2,shuffle = True,random_state = i+10)
#     X_train, X_valid, y_train, y_valid = GroupShuffleSplit(X, y,groups=train_input['pj_no'].array,test_size = 0.2,shuffle = True,random_state = i+10)

    # 上記のパラメータでモデルを学習する

    #'''
#     model = LGBMRegressorCV(
#                             n_estimators = 100,
#                             learning_rate = 0.1,
#                             n_trials = 100,
#                             n_iter_no_change=10,
#                             random_state=0,
#                             timeout = 60)
#     #'''

#     #ハイパーパラメータチューニングしない場合はこっち
#     #model = lgb.LGBMRegressor(random_state=10)

#     #学習
#     model.fit(X_train, y_train,categorical_feature=important_categorical_columns)

    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)
    # LightGBM parameters
    params = {
            'task' : 'train',
            'boosting_type' : 'dart',
            'objective' : 'regression',
            'metric' : {'mape'},
            'num_leaves' : 20,
            'learning_rate' : 0.05,
            'feature_fraction' : 0.5,
            'bagging_fraction' : 0.8,
            'bagging_freq': 5,
            'verbose' : 0
    }

    # train
    model = lgb.train(params,
                lgb_train,
                num_boost_round=4000,
                valid_sets=lgb_eval,
                early_stopping_rounds=300)

    #選ばれたハイパーパラメータの記録
    #temp_parameter_df = pd.DataFrame(model.best_params_.items()).set_index(0).T
    #Parameter_df = Parameter_df.append(temp_parameter_df )

    # 検証データを予測する
    #y_pred = model.predict(X_valid)
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    y_pred_train = model.predict(X_train, num_iteration=model.best_iteration)

    #検証スコア算出
    y_pred_df = pd.DataFrame(y_pred)
    y_valid = y_valid.reset_index().drop('index',axis=1)
    compared = pd.concat([y_pred_df,y_valid ],axis = 1)
    compared['APE'] = abs((compared[0]-compared['keiyaku_pr'])/compared['keiyaku_pr'])
    
    #検証スコア算出(train)
    y_pred_train_df = pd.DataFrame(y_pred_train)
    y_train = y_train.reset_index().drop('index',axis=1)
    compared_train = pd.concat([y_pred_train_df,y_train ],axis = 1)
    compared_train['APE'] = abs((compared_train[0]-compared_train['keiyaku_pr'])/compared_train['keiyaku_pr'])

    #検証スコアを記録
    temp_dict = {'MAPE':compared['APE'].mean()*100}
    temp_df = pd.DataFrame(temp_dict.items()).set_index(0).T
    MAPE_df = MAPE_df.append(temp_df)

    #検証スコアを記録
    temp_dict_train = {'MAPE':compared_train['APE'].mean()*100}
    temp_train_df = pd.DataFrame(temp_dict_train.items()).set_index(0).T
    MAPE_train_df = MAPE_train_df.append(temp_train_df)
    
    #予測を実行
    #y_test_pred = model.predict(X_test)
    y_test_pred = model.predict(X_test, num_iteration=model.best_iteration)
    y_test_pred_df = pd.DataFrame(y_test_pred)

    #提出用のデータフレーム作成
    temp_submit = pd.concat([test_data[['id']],y_test_pred_df],axis=1)
    temp_submit.rename(columns={0:'MAPE_'+str(i)},inplace = True)
    Submit_df = pd.merge(Submit_df,temp_submit,on='id')

    Submit_df['MAPE_sum'] = Submit_df['MAPE_sum'] + Submit_df['MAPE_'+str(i)]
    i = i+1
    
Submit_df['MAPE_mean'] = Submit_df['MAPE_sum']/(i)
    

0it [00:00, ?it/s]/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:762: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


[1]	valid_0's mape: 0.210199


/opt/conda/lib/python3.7/site-packages/lightgbm/callback.py:189: UserWarning: Early stopping is not available in dart mode
  warnings.warn('Early stopping is not available in dart mode')


[2]	valid_0's mape: 0.204587
[3]	valid_0's mape: 0.198705
[4]	valid_0's mape: 0.193528
[5]	valid_0's mape: 0.188679
[6]	valid_0's mape: 0.183718
[7]	valid_0's mape: 0.178653
[8]	valid_0's mape: 0.180442
[9]	valid_0's mape: 0.175592
[10]	valid_0's mape: 0.171162
[11]	valid_0's mape: 0.16743
[12]	valid_0's mape: 0.168712
[13]	valid_0's mape: 0.164748
[14]	valid_0's mape: 0.160957
[15]	valid_0's mape: 0.157256
[16]	valid_0's mape: 0.153904
[17]	valid_0's mape: 0.150862
[18]	valid_0's mape: 0.147859
[19]	valid_0's mape: 0.145417
[20]	valid_0's mape: 0.142751
[21]	valid_0's mape: 0.143571
[22]	valid_0's mape: 0.141173
[23]	valid_0's mape: 0.138661
[24]	valid_0's mape: 0.136362
[25]	valid_0's mape: 0.134274
[26]	valid_0's mape: 0.132243
[27]	valid_0's mape: 0.130289
[28]	valid_0's mape: 0.131194
[29]	valid_0's mape: 0.129433
[30]	valid_0's mape: 0.127562
[31]	valid_0's mape: 0.127793
[32]	valid_0's mape: 0.126103
[33]	valid_0's mape: 0.124646
[34]	valid_0's mape: 0.12329
[35]	valid_0's mape:

[528]	valid_0's mape: 0.0824739
[529]	valid_0's mape: 0.0820722
[530]	valid_0's mape: 0.0816864
[531]	valid_0's mape: 0.0820682
[532]	valid_0's mape: 0.0817129
[533]	valid_0's mape: 0.0820655
[534]	valid_0's mape: 0.0816919
[535]	valid_0's mape: 0.0813509
[536]	valid_0's mape: 0.0816708
[537]	valid_0's mape: 0.0812983
[538]	valid_0's mape: 0.0810278
[539]	valid_0's mape: 0.08137
[540]	valid_0's mape: 0.0817347
[541]	valid_0's mape: 0.0821674
[542]	valid_0's mape: 0.0818082
[543]	valid_0's mape: 0.0821568
[544]	valid_0's mape: 0.0825532
[545]	valid_0's mape: 0.0821033
[546]	valid_0's mape: 0.0817205
[547]	valid_0's mape: 0.081399
[548]	valid_0's mape: 0.0810913
[549]	valid_0's mape: 0.0814291
[550]	valid_0's mape: 0.081088
[551]	valid_0's mape: 0.0808285
[552]	valid_0's mape: 0.0805346
[553]	valid_0's mape: 0.0808215
[554]	valid_0's mape: 0.0811411
[555]	valid_0's mape: 0.0808753
[556]	valid_0's mape: 0.081231
[557]	valid_0's mape: 0.0815522
[558]	valid_0's mape: 0.081239
[559]	valid_0'

...

[2033]	valid_0's mape: 0.0690649
[2034]	valid_0's mape: 0.0690636
[2035]	valid_0's mape: 0.0690849
[2036]	valid_0's mape: 0.0690934
[2037]	valid_0's mape: 0.0690891
[2038]	valid_0's mape: 0.0691147
[2039]	valid_0's mape: 0.0691115
[2040]	valid_0's mape: 0.0691014
[2041]	valid_0's mape: 0.0690929
[2042]	valid_0's mape: 0.0690933
[2043]	valid_0's mape: 0.0691056
[2044]	valid_0's mape: 0.0691001
[2045]	valid_0's mape: 0.0691117
[2046]	valid_0's mape: 0.0691066
[2047]	valid_0's mape: 0.0690938
[2048]	valid_0's mape: 0.0690869
[2049]	valid_0's mape: 0.069084
[2050]	valid_0's mape: 0.0690834
[2051]	valid_0's mape: 0.0691066
[2052]	valid_0's mape: 0.0691147
[2053]	valid_0's mape: 0.0691224
[2054]	valid_0's mape: 0.0691414
[2055]	valid_0's mape: 0.0691662
[2056]	valid_0's mape: 0.0691512
[2057]	valid_0's mape: 0.069147
[2058]	valid_0's mape: 0.0691447
[2059]	valid_0's mape: 0.069146
[2060]	valid_0's mape: 0.0691127
[2061]	valid_0's mape: 0.0691178
[2062]	valid_0's mape: 0.0691262
[2063]	valid_

[2531]	valid_0's mape: 0.0684944
[2532]	valid_0's mape: 0.0684947
[2533]	valid_0's mape: 0.0684949
[2534]	valid_0's mape: 0.0684993
[2535]	valid_0's mape: 0.0684951
[2536]	valid_0's mape: 0.0684985
[2537]	valid_0's mape: 0.0685593
[2538]	valid_0's mape: 0.0685361
[2539]	valid_0's mape: 0.0685061
[2540]	valid_0's mape: 0.0685226
[2541]	valid_0's mape: 0.0684997
[2542]	valid_0's mape: 0.068488
[2543]	valid_0's mape: 0.0684966
[2544]	valid_0's mape: 0.0685113
[2545]	valid_0's mape: 0.0685271
[2546]	valid_0's mape: 0.0685459
[2547]	valid_0's mape: 0.068567
[2548]	valid_0's mape: 0.0685834
[2549]	valid_0's mape: 0.068609
[2550]	valid_0's mape: 0.0685619
[2551]	valid_0's mape: 0.0685838
[2552]	valid_0's mape: 0.0685596
[2553]	valid_0's mape: 0.0685084
[2554]	valid_0's mape: 0.0685257
[2555]	valid_0's mape: 0.0686421
[2556]	valid_0's mape: 0.0685765
[2557]	valid_0's mape: 0.0686087
[2558]	valid_0's mape: 0.0686449
[2559]	valid_0's mape: 0.0685794
[2560]	valid_0's mape: 0.0685274
[2561]	valid_

[3028]	valid_0's mape: 0.0684602
[3029]	valid_0's mape: 0.0684424
[3030]	valid_0's mape: 0.0684264
[3031]	valid_0's mape: 0.0684336
[3032]	valid_0's mape: 0.0684241
[3033]	valid_0's mape: 0.0684196
[3034]	valid_0's mape: 0.0684301
[3035]	valid_0's mape: 0.068452
[3036]	valid_0's mape: 0.0684355
[3037]	valid_0's mape: 0.0684223
[3038]	valid_0's mape: 0.0683948
[3039]	valid_0's mape: 0.0683845
[3040]	valid_0's mape: 0.0683919
[3041]	valid_0's mape: 0.0683787
[3042]	valid_0's mape: 0.0683682
[3043]	valid_0's mape: 0.0683811
[3044]	valid_0's mape: 0.0683698
[3045]	valid_0's mape: 0.0683575
[3046]	valid_0's mape: 0.068364
[3047]	valid_0's mape: 0.0683827
[3048]	valid_0's mape: 0.0683986
[3049]	valid_0's mape: 0.0683899
[3050]	valid_0's mape: 0.0683942
[3051]	valid_0's mape: 0.068407
[3052]	valid_0's mape: 0.0684224
[3053]	valid_0's mape: 0.0684079
[3054]	valid_0's mape: 0.0684362
[3055]	valid_0's mape: 0.0684432
[3056]	valid_0's mape: 0.0684542
[3057]	valid_0's mape: 0.0684362
[3058]	valid_

[3524]	valid_0's mape: 0.0684292
[3525]	valid_0's mape: 0.0684181
[3526]	valid_0's mape: 0.0684089
[3527]	valid_0's mape: 0.0684212
[3528]	valid_0's mape: 0.068415
[3529]	valid_0's mape: 0.068407
[3530]	valid_0's mape: 0.0684317
[3531]	valid_0's mape: 0.0684494
[3532]	valid_0's mape: 0.0684643
[3533]	valid_0's mape: 0.0684537
[3534]	valid_0's mape: 0.0684436
[3535]	valid_0's mape: 0.0684511
[3536]	valid_0's mape: 0.0684407
[3537]	valid_0's mape: 0.0684521
[3538]	valid_0's mape: 0.0684791
[3539]	valid_0's mape: 0.0684847
[3540]	valid_0's mape: 0.0685115
[3541]	valid_0's mape: 0.0685328
[3542]	valid_0's mape: 0.0685452
[3543]	valid_0's mape: 0.0685669
[3544]	valid_0's mape: 0.0685862
[3545]	valid_0's mape: 0.0685677
[3546]	valid_0's mape: 0.0685494
[3547]	valid_0's mape: 0.0685753
[3548]	valid_0's mape: 0.0685879
[3549]	valid_0's mape: 0.068571
[3550]	valid_0's mape: 0.0685748
[3551]	valid_0's mape: 0.0685613
[3552]	valid_0's mape: 0.0685699
[3553]	valid_0's mape: 0.0685623
[3554]	valid_

5it [12:55, 158.15s/it]


In [0]:
#ハイパーパラメータ確認
Parameter_df

""


In [0]:
#各回でのMAPE検証値
MAPE_df

,MAPE
1,7.227133
1,6.765057
1,6.659730
1,6.998840
1,6.841963


In [0]:
MAPE_train_df

,MAPE
1,2.668686
1,2.682427
1,2.694609
1,2.717653
1,2.684807


In [0]:
compared['APE'].describe()

count    1291.000000
mean        0.068420
std         0.064444
min         0.000019
25%         0.026179
50%         0.054041
75%         0.098001
max         1.204453
Name: APE, dtype: float64

In [0]:
compared_train.sort_values('APE',ascending=False)

,0,keiyaku_pr,APE
3722,1.350937e+07,10800000,0.250867
1469,4.339854e+06,3500000,0.239958
1527,8.916261e+06,7300000,0.221406
2885,2.895729e+07,24000000,0.206554
537,1.597706e+07,19900000,0.197132
3718,1.967153e+07,17000000,0.157149
3719,1.967153e+07,17000000,0.157149
1712,2.503341e+07,29500000,0.151410
4832,1.834554e+07,16000000,0.146597
3498,1.945338e+07,17000000,0.144316


In [0]:
compared.sort_values('APE',ascending=False)

,0,keiyaku_pr,APE
1109,1.212449e+07,5500000,1.204453
435,2.666506e+07,18500000,0.441355
926,1.089518e+07,7900000,0.379137
963,8.104393e+06,6000000,0.350732
638,1.600976e+07,11990000,0.335260
486,2.123418e+07,31600000,0.328032
1097,2.652817e+07,19990000,0.327072
26,2.632706e+07,20300000,0.296900
578,5.893033e+07,46300000,0.272793
1201,2.975073e+07,23700000,0.255305


In [0]:
MAPE_df.mean()

0
MAPE    6.898545
dtype: float64

In [47]:
#結果の確認
#Submit_df 

In [0]:
importance_df = pd.DataFrame([model.feature_importance(importance_type='gain'),X.columns]).T
importance_df.rename(columns={0:'importance',1:'feature_name'},inplace =True)
importance_df.set_index('feature_name',inplace=True)

In [0]:
importance_df.sort_values('importance',ascending = False)

,importance
feature_name,
koji_tatemenseki,1.43817e+18
koji_totimenseki,8.07062e+17
rosenka_hb,7.67261e+17
kijun_tatemenseki,7.58759e+17
kijun_totimenseki,3.40724e+17
tt_mseki,3.16716e+17
toho_kyori1,2.14758e+17
latitude,1.95784e+17
eki_nm1,1.95104e+17


In [0]:
Submit_df[['id','MAPE_mean']].to_csv('output/submit_0803_4.tsv',header=False,index=False,sep ='\t')

In [0]:
importance = importance_df.reset_index().rename(columns={'feature_name':'項目名'})

In [0]:
#pd.merge(importance,data_definition).sort_values('importance',ascending=False)

,項目名,importance,データ種別,項目名（日本語）,説明,データ型
60,rosenka_hb,7.67261e+17,現場,路線価格(平米),路線価格㎡単価を収録,数値
1,tt_mseki,3.16716e+17,号棟,建物面積,建物面積㎡数,数値
123,eki_nm1,1.95104e+17,現場,駅1,主に利用する駅の名前,テキスト
62,kijun_hb,1.8379e+17,現場,基準地価(平米),基準地価㎡単価を収録,数値
5,magutchi,1.78157e+17,号棟,間口〔m〕,前面道路の間口〔m〕,数値
61,koji_hb,1.72052e+17,現場,公示地価(平米),公示地価㎡単価を収録,数値
65,tt_mseki_min_hb,1.01132e+17,現場,建物最小面積(平米),建物最小面積㎡数,数値
67,tc_mseki_avg_hb,9.03218e+16,現場,土地平均面積(平米),土地平均面積㎡数,数値
0,tc_mseki,7.48336e+16,号棟,土地面積,土地面積㎡数,数値
66,tt_mseki_max_hb,5.65857e+16,現場,建物最大面積(平米),建物最大面積㎡数,数値
